In [2]:
import pickle
import json
import matplotlib.pyplot as plt
import pymongo
import tweepy
import cnfg
import pandas as pd
import requests
from bson.son import SON
from pymongo import MongoClient
from sklearn.feature_extraction.text import CountVectorizer
from requests_oauthlib import OAuth1


%matplotlib inline

In [3]:
client = MongoClient(port=27017)
db = client.tweet_on_vegan
vegan_tweets = db.streaming_tweets

In [4]:
config = cnfg.load("/home/ubuntu/Documents/.twitter_config")

oauth = OAuth1(config["consumer_key"],
               config["consumer_secret"],
               config["access_token"],
               config["access_token_secret"])

auth = tweepy.AppAuthHandler(config["consumer_key"],
                           config["consumer_secret"])
#auth.set_access_token(config["access_token"],
#                      config["access_token_secret"])
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
for tweet in vegan_tweets.find({}).skip(116500):
    id_ = tweet['id']
    if tweet['text'][:2] == "RT":
        try:
            extended_tweet = api.get_status(id_, tweet_mode="extended")
        except:
            continue
            
        retweet = 1
        try:
            full_text = extended_tweet._json['retweeted_status']['full_text']          
        except:
            try: 
                full_text = extended_tweet._json['full_text']
            except:
                continue
        vegan_tweets.update({"_id": tweet["_id"]}, 
                            {"$set": {"text": full_text, "is_retweet": retweet}})
    else:
        retweet = 0
        vegan_tweets.update({"_id": tweet["_id"]}, 
                            {"$set": {"is_retweet": retweet}})



/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


Rate limit reached. Sleeping for: 826
Rate limit reached. Sleeping for: 830
Rate limit reached. Sleeping for: 831
Rate limit reached. Sleeping for: 831
Rate limit reached. Sleeping for: 828


In [ ]:
extended_tweet = api.get_status(id_, tweet_mode="extended")